<h1> High Quality Momentum Trading Strategy </h1>

Calculating the number of stocks to be bought for the top 20 High Quality Momentum stocks in the S&P 500 Index for a user specific portfolio.

In [15]:
#Importing libraries

import numpy as np 
import pandas as pd 
import requests
from scipy import stats
import math

#Importing the constituents of the S&P 500

stocks= pd.read_csv('constituents.csv')

#Viewing the constituents

stocks.head()

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [16]:
# Acquiring the IEX API Token
from secrets import IEX_CLOUD_API_TOKEN

<h1> Making API Calls and Creating our dataframe</h1>

In [17]:
#We use batch API calls here to improbbe the performance of our code
#reference https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One Year Price Return', 'Number of Shares to Buy']


<h1>High Quality Momentum Trading strategy</h1>

To have an effective Moment Trading Strategy, we need to ensure that the stocks we plan to invest in are High Quality Momentum stocks. High Quality Momentum or HQM stocks are preffered over Low Quality Momentum stocks as they show a steady growth momentum over a larger period of time and are not triggered by rare and specific incidents.

In [18]:
%%capture
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Stocks', 
                'One Year Price Return', 
                'One Year Return Percentile',
                'Six Month Price Return',
                'Six Month Return Percentile',
                'Three Month Price Return',
                'Three Month Return Percentile',
                'One Month Price Return',
                'One Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
                            


<h1>Calculating the percentile scores</h1>
In this part, we will be calculating the momentum percentiles of the enlisted stocks over the periods of:
<ul>
<li>One year price return</li>
<li>Six month price return</li>
<li>Three month price return</li>
<li>One month price return</li>
</ul>

In [19]:
time_periods = [
                'One Year',
                'Six Month',
                'Three Month',
                'One Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
   
hqm_dataframe

,Ticker,Price,Number of Stocks,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,149.83,N/A,-0.241362,0.12,-0.178045,0.33,-0.147997,0.17,-0.043016,0.54,N/A
1,AOS,61.12,N/A,-0.062843,0.43,-0.013354,0.58,-0.170743,0.12,-0.063734,0.46,N/A
2,ABT,121.82,N/A,-0.02648,0.47,-0.08215,0.5,-0.027486,0.44,-0.015301,0.72,N/A
3,ABBV,162.62,N/A,0.479465,0.96,0.501638,0.99,0.176867,0.92,-0.015222,0.73,N/A
4,ABMD,292.65,N/A,-0.187978,0.23,-0.204077,0.26,-0.000548,0.53,-0.131691,0.22,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,144.89,N/A,-0.089889,0.39,-0.136627,0.38,-0.117826,0.25,-0.023325,0.66,N/A
96,CNC,84.07,N/A,0.326862,0.9,0.14637,0.81,0.05989,0.73,-0.058972,0.48,N/A
97,CNP,32.62,N/A,0.351912,0.93,0.213138,0.9,0.156163,0.87,0.030288,0.84,N/A
98,CDAY,58.06,N/A,-0.429788,0.01,-0.553381,0.01,-0.190622,0.08,-0.175733,0.12,N/A


<h1>Calculating the HQM Score</h1> 

The High Quality Momentum score is the arithmetic mean of the afforementioned four momentum percentiles.

In [20]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

# Selecting our top 20 HQM stocks

hqm_dataframe.sort_values(by = 'HQM Score', ascending= True)
hqm_dataframe = hqm_dataframe[:21]


<h1> Accepting the desired portfolio amount and issuing reccomendations</h1>

Here, we accept the user's amount to be invested and return the number of stocks the user should buy of each respective High Quality Momentum Stock.


In [21]:
# Asking for the value of portfolio

def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

#Calculating the number of stocks

position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Stocks'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe




1000000


,Ticker,Price,Number of Stocks,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,149.83,317,-0.241362,0.12,-0.178045,0.33,-0.147997,0.17,-0.043016,0.54,0.29
1,AOS,61.12,779,-0.062843,0.43,-0.013354,0.58,-0.170743,0.12,-0.063734,0.46,0.3975
2,ABT,121.82,390,-0.02648,0.47,-0.08215,0.5,-0.027486,0.44,-0.015301,0.72,0.5325
3,ABBV,162.62,292,0.479465,0.96,0.501638,0.99,0.176867,0.92,-0.015222,0.73,0.9
4,ABMD,292.65,162,-0.187978,0.23,-0.204077,0.26,-0.000548,0.53,-0.131691,0.22,0.31
5,ACN,327.22,145,0.058355,0.64,-0.143792,0.37,-0.092263,0.33,-0.074495,0.42,0.44
6,ATVI,78.2,608,-0.168515,0.25,-0.026912,0.54,-0.031167,0.43,-0.038858,0.57,0.4475
7,ADM,93.7,508,0.567805,0.98,0.487342,0.98,0.275536,1.0,0.004788,0.76,0.93
8,ADBE,429.26,110,-0.237235,0.13,-0.390343,0.06,-0.197958,0.07,-0.079497,0.4,0.165
9,AAP,219.1,217,0.169423,0.73,-0.012697,0.59,0.006514,0.57,0.044093,0.92,0.7025
